# Лабораторна робота 1 з "Методів криптоаналізу"
## Тема: Баєсiвський пiдхiд в криптоаналiзi: побудова i дослiдження детермiнiстичної та стохастичної вирiшуючих функцiй.

**Виконали**\
Дигас Богдан, ФІ-52мн\
Юрчук Олексій, ФІ-52мн

In [16]:
import pandas as pd

df = pd.read_csv('.\\var_data\\prob_15.csv')
for index, row in df.iterrows():
    print(f"\nRow {index + 1} data:")
    for col_name, value in row.items():
        print(f"  {col_name}: {value}")

[0.14, 0.14, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]
